In [ ]:
#Muhammad Hasham Ul Haq
#i200752@nu.edu.pk
%pip install transformers pandas numpy librosa datasets torch

In [ ]:
import pandas as pd
import numpy as np
import json
import librosa
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC, TrainingArguments, Trainer, Wav2Vec2CTCTokenizer
from datasets import Dataset
import torch
from torch.quantization import quantize_dynamic

vocab_list = {"ن": 0, "ر": 1, "ٸ": 2, "f": 3, "b": 4, "l": 6, "ح": 7, "خ": 8, "ع": 9, "ش": 10, "س": 11, "0": 12, "إ": 13, "ف": 14, "و": 15, "ی": 16, "۱": 17, "‌": 18, "ۃ": 19, "i": 20, "ڑ": 21, "ل": 22, "2": 23, "3": 24, "۰": 25, "ۓ": 26, "۲": 27, "a": 28, "ک": 29, "‬": 30, "ت": 31, "v": 32, "d": 33, "ﷲ": 34, "۵": 35, "r": 36, "‫": 37, "h": 38, "پ": 39, "ژ": 40, "w": 41, "‎": 42, "‍": 43, "u": 44, "s": 45, "ب": 46, "ث": 47, "1": 48, "ض": 49, "۶": 50, "گ": 51, "ۂ": 52, "ں": 53, "ك": 54, "ط": 55, "t": 56, "ھ": 57, "5": 58, "7": 59, "غ": 60, "ـ": 61, "ء": 62, "ظ": 63, "ي": 64, "8": 65, "c": 66, "⁦": 67, "ص": 68, "e": 69, "­": 70, "۸": 71, "۳": 72, "6": 73, "m": 74, "ٹ": 75, "چ": 76, "آ": 77, "ﮟ": 78, "ا": 79, "‏": 80, "ئ": 81, "g": 82, "o": 83, "۷": 84, " ": 85, "4": 86, "۴": 87, "د": 88, "​": 89, "ق": 90, "ڈ": 91, "k": 92, "p": 93, "ذ": 94, "ج": 95, "ؤ": 96, "ہ": 97, "ز": 98, "n": 99, "`": 100, "x": 101, "۹": 102, "ه": 103, "ے": 104, "9": 105, "م": 106, "|": 5, "[UNK]": 107, "[PAD]": 108}

In [ ]:
# Load your CSV file
data = pd.read_csv('cleaned_file.csv')  # Replace with your CSV file path

# Preprocess transcripts if needed
def preprocess_text(text):
    text = text.lower().strip()
    return text

data['transcript'] = data['transcript'].apply(preprocess_text)


In [ ]:
# Import the necessary libraries
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor

# Create a processor using your vocabulary
vocab_dict = {v: k for k, v in enumerate(vocab_list)}  # Assume vocab_list is already defined
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

tokenizer = Wav2Vec2CTCTokenizer("vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

# Load the audio files and preprocess transcripts
def load_audio(file_name):
    file_path = f'/content/drive/MyDrive/UrduRecordings.zip (Unzipped Files)/Train/{file_name}'  # Update with the correct path
    audio, sampling_rate = librosa.load(file_path, sr=16000)
    return audio

data['audio'] = data['audio_File'].apply(load_audio)
data = data[['transcript', 'audio']]


In [ ]:
# Convert DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(data)


In [ ]:
# Define the preparation function
def prepare_dataset(batch):
    audio = batch["audio"]
    batch["input_values"] = processor(audio, sampling_rate=16000).input_values
    with processor.as_target_processor():
        batch["labels"] = processor(batch["transcript"]).input_ids
    return batch

dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names)

# Define the data collator
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)


In [ ]:
# Split the dataset if needed
train_test_split = dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']


In [ ]:
# Load the model
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53",
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

# Freeze the feature encoder
model.freeze_feature_encoder()


In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./urdu_transefer_learning",
    group_by_length=True,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    evaluation_strategy="steps",
    num_train_epochs=30,
    fp16=True,
    save_steps=100,
    eval_steps=100,
    logging_steps=10,
    learning_rate=3e-4,
    warmup_steps=500,
    save_total_limit=2,
)


In [ ]:
# Start training
trainer.train()


In [ ]:

#code for quantization of the model
trained_model.eval()
quantized_model = quantize_dynamic(
    trained_model,
    {torch.nn.LSTM, torch.nn.Linear},
    dtype=torch.qint8
)
torch.save(quantized_model.state_dict(), 'quantized_model.pth')